In [15]:
#| default_exp structs

In [1]:
#| export
from typing import List, Optional, Dict, Callable, Any

import jax
import jax.numpy as jnp

from typing import Tuple
from numpy.random import default_rng
import msprime
import tskit
import numpy as np
import random
from collections import defaultdict
from flax.struct import dataclass as flax_dataclass, field

#testing
import jax
import jax.numpy as jnp
from fastcore.test import test_eq, test_ne


import jax.numpy as jnp
from flax.struct import dataclass as flax_dataclass, field
from typing import List, Optional, TYPE_CHECKING
from typing import Optional, TYPE_CHECKING

# No need for cached_property anymore for these core attributes
# from functools import cached_property

import jax.numpy as jnp
from flax.struct import dataclass as flax_dataclass, field
from typing import List, Optional, TYPE_CHECKING

In [2]:
#| export
# chewc/structs.py


@flax_dataclass(frozen=True)
class SimParam:
    """
    Global, run-constant parameters & metadata.
    Keep only JAX-compatible fields (arrays, tuples, scalars).
    """
    gen_map: jnp.ndarray           # (n_chr, loci_per_chr or ragged handled upstream)
    ploidy: int
    traits: Optional[Any] = None   # e.g., TraitCollection
    recomb_params: tuple = (2.6, 0.0, 0.0)
    var_e: Optional[jnp.ndarray] = None

@flax_dataclass(frozen=True)
class Population:
    """
    User-facing container for population data (setup & analysis).
    The JIT carry uses SimState (internal) instead.
    """
    # Core genotype/IBD
    geno: jnp.ndarray              # (max_pop, n_chr, ploidy, n_loci)
    ibd: jnp.ndarray               # same shape as geno

    # IDs & pedigree
    id: jnp.ndarray                # (max_pop,)
    iid: jnp.ndarray               # (max_pop,)
    mother: jnp.ndarray            # (max_pop,)
    father: jnp.ndarray            # (max_pop,)
    sex: jnp.ndarray               # (max_pop,)
    gen: jnp.ndarray               # (max_pop,)

    # Phenotypes & flags
    pheno: jnp.ndarray             # (max_pop, n_traits)
    fixEff: jnp.ndarray            # (max_pop, n_fixeff) or (max_pop,)
    is_active: jnp.ndarray         # (max_pop,) bool

    # Optional genetic values
    gv: Optional[jnp.ndarray] = None
    bv: Optional[jnp.ndarray] = None
    dd: Optional[jnp.ndarray] = None
    aa: Optional[jnp.ndarray] = None
    ebv: Optional[jnp.ndarray] = None

    # Non-pytree metadata
    misc: Optional[Dict[str, Any]] = field(default=None, pytree_node=False)
    miscPop: Optional[Dict[str, Any]] = field(default=None, pytree_node=False)

    @property
    def nInd(self) -> int:
        # host-side convenience; fine if not used in JIT
        return int(jnp.sum(self.is_active))

In [3]:
#| hide
import nbdev; nbdev.nbdev_export()